In [ ]:
using Random
using Plots

In [ ]:
# Constants
my_count = 1
n_values = [10^i for i in 4:7]  # Vectors Sizes
q_values = [10^i for i in 2:5]  # Amount of Keys

# Initialize the list of vectors and keys
vector_list = [rand(1:10^5, n) for n in n_values]
key_list = [collect(1:q) for q in q_values]  # K2[1-100], ..., K5[1-10^5]

# Initialize the list of timers
time_simple_search = zeros(length(n_values), length(q_values))
time_optimized_search = zeros(length(n_values), length(q_values))
time_binary_search = zeros(length(n_values), length(q_values))
time_sort = zeros(length(n_values))

In [ ]:
# Simple linear search function
function simple_search(vector, key)
    for i in 1:length(vector)
        if key == vector[i]
            return i  # Returns the index where the key was found
        end
    end
    return -1  # Returns -1 if the key is not found
end

In [ ]:
# Optimized linear search function
function optimized_search(vector, key)
    for i in 1:length(vector)
        if key == vector[i]
            return i  # Returns the index where the key was found
        elseif key < vector[i]
            return -1  # Returns -1 if the key is not found
        end
    end
    return -1  # Returns -1 if the key is not found
end

In [ ]:
# Binary search function
function binary_search(vector, key)
    low, high = 1, length(vector)
    while low <= high
        mid = div(low + high, 2)
        if vector[mid] == key
            return mid  # Returns the index where the key was found
        elseif vector[mid] < key
            low = mid + 1
        else
            high = mid - 1
        end
    end
    return -1  # Returns -1 if the key is not found
end

In [ ]:
# Dummy search function
function dummy_search(search_func, vector, keys)
    [search_func(vector, k) for k in keys]
end

In [ ]:
# Benchmark macro
function my_benchmark(search_func, time_matrix)
    for i in 1:length(vector_list)   # For each vector size
        for j in 1:length(key_list)  # For each key numbers
            start = time_ns()
            for c in 1:my_count
                dummy_search(search_func, [vector_list[i]], [key_list[j]])
            end
            time_matrix[i, j] = (time_ns() - start) / 1e9
        end
    end
end

In [ ]:
my_benchmark(simple_search, time_simple_search)

In [ ]:
# Sorting the vectors - Benchmarking
for i in 1:length(vector_list)   # For each vector size
    start = time_ns()
    for c in 1:my_count
        vector_list[i] = sort(vector_list[i])
    end
    time_sort[i] = (time_ns() - start) / 1e9
end

vector_list

In [ ]:
my_benchmark(optimized_search, time_optimized_search)

In [ ]:
my_benchmark(binary_search, time_binary_search)

In [ ]:
# Plotting the results
xtick_positions = [0, 25000, 50000, 75000, 10^5]
xtick_labels = ["0", "25.000", "50.000", "75.000", "100.000"]

for i in 1:length(n_values)
    p = plot(
        q_values,
        time_simple_search[i, :],
        label = "Simples",
        xlabel = "Número de Busca (q)",
        ylabel = "Tempo (s)",
        title = "Tamanho do Vetor (n) = $(n_values[i])",
        legend = :topleft,
        xticks = (xtick_positions, xtick_labels),
        yscale = :log10,  # log scale
        lw = 2
    )
    plot!(p, q_values, time_optimized_search[i, :], label = "Otimizada", lw = 2)
    plot!(p, q_values, time_binary_search[i, :], label = "Binária", lw = 2)
    plot!(p, q_values, time_sort[i] .* ones(length(q_values)), label = "Ordenação", lw = 2)

    display(p)
    savefig(p, "../imgs/log_10^$(i+3).png")
end